In [40]:
# CNN Tensor flow Example
# This is code is following 'https://github.com/minsuk-heo/deeplearning/blob/master/src/CNN_Tensorflow.ipynb'
import tensorflow as tf
# load data from keras
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# the last 10000 dataset set aside for validation 
x_val = x_train[50000:60000]
y_val = y_train[50000:60000]
# total 50000 data sets are used for training
x_train = x_train[0:50000]
y_train = y_train[0:50000]

In [41]:
# Reshape
import numpy as np

x_train = np.reshape(x_train, (50000, 28, 28, 1))
x_val = np.reshape(x_val, (10000, 28, 28, 1))
x_test = np.reshape(x_test, (10000, 28, 28, 1))


In [42]:
# Normalize

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255

x_train /= 255
x_val /=255
x_test /=255


In [43]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [44]:
# Define initial value generating function
def weighted_variable(shape) :
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape) :
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [45]:
# Define filter function
def conv2d(x, W) :
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# Define pooling function
def max_pool_2x2(x) :
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,1,1,1], padding='SAME')

In [46]:
# Define first convolution layer. 5 x 5 x 1 x 16 16filters applied
W_conv1 = weighted_variable([5,5,1,16])
b_conv1 = bias_variable([16])
# Define activation function. We use relu function
h_conv1 = tf.nn.relu( conv2d(x, W_conv1) + b_conv1)
# pooling layer
h_pool1 = max_pool_2x2(h_conv1)

In [47]:
# Define second convolution layer. 5 x 5 x 16 x 32, 32 filters applied
W_conv2 = weighted_variable([5, 5, 16, 32])
b_conv2 = bias_variable([32])
# Define activation function, we use relu function
h_conv2 = tf.nn.relu( conv2d(h_pool1, W_conv2) + b_conv2)
# pooling layer
h_pool2 = max_pool_2x2(h_conv2)

In [48]:
# Define fully connected layer

# First hidden layer 128 
W_fc1 = weighted_variable([7*7*32, 128])
b_fc1 = bias_variable([128])
# Flatting h_pool1
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32])
h_fc1 = tf.nn.relu( tf.matmul( h_pool2_flat, W_fc1) + b_fc1 )

In [49]:
# second hidden layer 10
W_fc2 = weighted_variable([128, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.relu( tf.matmul(h_fc1, W_fc2) + b_fc2)

In [50]:
#Loss function : cross-entropy
cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv ) )

In [51]:
# Optimizer : Adam
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

In [52]:
# Define accuracy and correctness
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [55]:
# Train and Test
init = tf.global_variables_initializer()

epoch_cnt = 3
batch_size = 500
iteration = len(x_train) // atch_size

with tf.Session() as sess :
    tf.set_random_seed(7777)
    sess.run(init)
    
    for epoch in range(epoch_cnt) :
        avg_loss = 0
        start = 0
        end = batch_size
        
        for i in range(iteration):
            if i%10 == 0:
                train_accuracy = accuracy.eval(feed_dict={x:x_train[start: end], y_: y_train[start: end]})
                print("step "+ str(i) + ": training accuracy: "+str(train_accuracy))
            train_step.run(feed_dict={x:x_train[start: end], y_: y_train[start: end]})
            start += batch_size; end += batch_size    
        
        # Validate model
        val_accuracy = accuracy.eval(feed_dict={x:x_val, y_: y_val})
        print("validation accuracy: "+str(val_accuracy))
        
    test_accuracy = accuracy.eval(feed_dict={x:x_test, y_: y_test}) 
    print("test accuracy: "+str(test_accuracy))

ValueError: Cannot feed value of shape (500,) for Tensor u'Placeholder_4:0', which has shape '(?, 10)'